In [1]:
# def get_file_counts(filename):
#     with open(filename) as fp:
#         count = 0
#         for _ in fp:
#             count += 1
#     return count

file_name_list = ['2015_32384541_32M-006.csv', '2016_31758353_31M-007.csv', '2017_24987298_24M-004.csv',\
                  '2018_20731058_20M-005.csv', '2019_16476247_16M-003.csv', '2020_3888214_3M.csv', '2021_2576772_2M.csv']


# file_counts = []

# for i in file_name_list:
#     file_counts.append(get_file_counts(i))

# print(file_counts)

file_counts = [32384542, 31758354, 24987299, 20731059, 16476248, 3888215, 2576773]
total_size = sum(file_counts)
print(total_size)

132802490


In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [3]:
def my_generator(chunks):
    global file_name_list
    while True:
        for i in range(2, 4):
            f = file_name_list[i]
#             f = file_name_list[4]
            for data in pd.read_csv(f, chunksize=chunks):
                data = data[data['Trip Seconds'].notna()]
                data = data[data['Trip Miles'].notna()]
                data = data[data['Fare'].notna()]
                data = data[data['Trip Start Timestamp'].notna()]
                data['month'] = pd.to_datetime(data['Trip Start Timestamp']).dt.month
                x = data[['Trip Seconds', 'Trip Miles', 'month']].to_numpy()
                y = data['Fare'].to_numpy()
                yield x, y
            
def validator_generator(chunks):
    while True:
        for data in pd.read_csv('2021_2576772_2M.csv', chunksize=chunks):
            data = data[data['Trip Seconds'].notna()]
            data = data[data['Trip Miles'].notna()]
            data = data[data['Fare'].notna()]
            data = data[data['Trip Start Timestamp'].notna()]
            data['month'] = pd.to_datetime(data['Trip Start Timestamp']).dt.month
            x = data[['Trip Seconds', 'Trip Miles', 'month']].to_numpy()
            y = data['Fare'].to_numpy()
            yield x, y

In [4]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(16, kernel_initializer='normal',input_dim = 3, activation='relu'))

# The Hidden Layer :
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


In [5]:
file_length = file_counts[2] + file_counts[3]
chunksize = 15000

NN_model.fit_generator(my_generator(chunksize),
          steps_per_epoch=file_length//chunksize,
          epochs=3,
          verbose=2,
          validation_data=validator_generator(chunksize),
          validation_steps=50
            )

C:\Users\Pulak\AppData\Local\Temp/ipykernel_3304/3312326068.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  NN_model.fit_generator(my_generator(chunksize),


Epoch 1/3
3047/3047 - 1324s - loss: 2.1603 - mean_absolute_error: 2.1603 - val_loss: 3.7293 - val_mean_absolute_error: 3.7293 - 1324s/epoch - 434ms/step
Epoch 2/3
3047/3047 - 1322s - loss: 1.5723 - mean_absolute_error: 1.5723 - val_loss: 3.7142 - val_mean_absolute_error: 3.7142 - 1322s/epoch - 434ms/step
Epoch 3/3
3047/3047 - 1321s - loss: 1.5142 - mean_absolute_error: 1.5142 - val_loss: 4.2288 - val_mean_absolute_error: 4.2288 - 1321s/epoch - 434ms/step


In [10]:
def predict_fare(time, distance, month):
    return NN_model.predict([[time, distance, month]])[0][0]


predict_fare(1000, 0.5, 7)

13.143008

In [11]:
NN_model.save('NN_model')

INFO:tensorflow:Assets written to: NN_model\assets
